In [3]:
from utils import *


# Функция маркирует в тексте ошибки красным
def highlight_by_indices(text, intervals):
    HIGHLIGHT_START = "\033[91m"  # красный
    HIGHLIGHT_END = "\033[0m"

    intervals = sorted(intervals, key=lambda x: x[0])
    result = ""
    last_index = 0
    for start, end in intervals:
        result += text[last_index:start]
        result += f"{HIGHLIGHT_START}{text[start:end]}{HIGHLIGHT_END}"
        # result += f"[ERROR]{text[start:end]}[ERROR]"
        last_index = end
    result += text[last_index:]
    return result



# Класс для маркировки ошибок в тексте
# Пайплайн простой:
# 1) Делаем правильное решение задачи
# 2) Разделяем на шаги решение сгенерированое нейронкой
# 3) Разделяем на шаги решение которое прислал пользователь
# 4) С помощью тегов просим нейронку найти ошибки в тексте и сразу же их парсим, получая символьные диапазоны
# 5) Выделяем ошибки с помощью тегов HIGHLIGHT
class WebMarkingError:
    def __init__(self, prompts, batch_size=10, model_name="Qwen/Qwen3-4B-Thinking-2507"):
        self.ask_llm = ask_llm
        self.prompts = prompts
        self.model_name = model_name
        self.batch_size = batch_size

    # 1) Делаем правильное решение задачи
    def solve_task(self, task):
        results = rerun_until_filled(self.model_name,
                                     [task],
                                     show_progress=True,
                                     title='Make solutions')
        return results[0]

    # 3) Разделяем на шаги решение которое прислал пользователь
    def decompose_solutions(self, text):
        dec_prompts = [self.prompts['decompose'].replace('{SOLUTION}', text.replace('\n', '  '))]
        results = rerun_until_filled(
            self.model_name,
            dec_prompts,
            show_progress=True,
            title='',
            texts_for_decompose=[text]
        )
        return list(results[0].keys()), list(results[0].values())  # steps, indexes

    # 2) Разделяем на шаги решение сгенерированое нейронкой
    def decompose_our_solutions(self, solution):
        dec_prompt = [self.prompts['decompose'].replace('{SOLUTION}', solution.replace('\n', '  '))]

        results = rerun_until_filled(
            self.model_name.replace('Thinking', 'Instruct'),
            dec_prompt,
            show_progress=True,
            title='Decompose tasks',
            # texts_for_decompose=solution
        )
        # print(results)
        # steps = re.findall(r'''\d+\.\s[\"\' ]*(.*?)[\"\' ]*(?=\n\d+\.|$)''', results[0], flags=re.S)
        return results[0] # '\n'.join(f"{i + 1}. {x}" for i, x in enumerate(steps))

    def get_prompt(self, task, steps, steps_our_solution):
        prompt = make_prompts(task, steps, steps_our_solution, self.batch_size)
        p = []
        for i in prompt:
            p += [i]
        return p

    def group_marking(self, all_responses, solution, indexes):
        pattern = r"\d+\.\s(.*?)(?=\n\d+\.|$)"
        joined = "\n".join(all_responses)
        ans_steps = re.findall(pattern, joined, flags=re.S)
        return extract(ans_steps, indexes, solution)

    # 4) С помощью тегов просим нейронку найти ошибки в тексте
    def find_errors(self, task, steps, indexes, steps_our_solution, solution):
        self.get_prompt(task, steps, steps_our_solution)
        all_responses = rerun_until_filled(
            self.model_name,
            self.get_prompt(task, steps, steps_our_solution),
            show_progress=True,  # пусть покажет общий прогресс
            title='Errors markering'
        )
        # и сразу же их парсим, получая символьные диапазоны
        final_result = self.group_marking(all_responses, solution, indexes)
        return final_result

    def inference(self, task, solution):
        our_sol = self.solve_task(task)
        dec_our_sol = self.decompose_our_solutions(our_sol)
        steps, indexes = self.decompose_solutions(solution)
        return self.find_errors(task, steps, indexes, dec_our_sol, solution), self.mark(task), self.hints(task, '\n'.join(steps), dec_our_sol)

    def mark(self, task):
        res = ask_llm(mark.replace('TASK', task), self.model_name)
        while len(res) != 1:
            res = ask_llm(mark.replace('TASK', task), self.model_name)
        return res

    def hints(self, task, sol, sol_c):
        pattern = r"Подсказка\s*\d+\s*(.*?)(?=Подсказка\s*\d+|$)"
        res = ask_llm(hints.replace('{task}', task).replace('{correct_solution}', sol_c).replace('{wrong_solution}', sol), self.model_name)
        matches = re.findall(pattern, res, flags=re.DOTALL | re.IGNORECASE)
        res = [m.strip() for m in matches if m.strip()]
        while len(res) != 3:
            res = ask_llm(hints.replace('{task}', task).replace('{correct_solution}', sol_c).replace('{wrong_solution}', sol), self.model_name)
            matches = re.findall(pattern, res, flags=re.DOTALL | re.IGNORECASE)
            res = [m.strip() for m in matches if m.strip()]
        return res

        # 5) Выделяем ошибки с помощью тегов HIGHLIGHT
    def __call__(self, task, solution):
        dia, hints, mark = self.inference(task, solution)
        return dia, hints, mark

# Пример как получить диапазоны символов с ошибкой

## Ставим промт для декомпозиции
```python
prompts = {'decompose':prompt_decompose_solution}
```

## Делаем экземпляр класса
```python
web = WebMarkingError(prompts)
```

## Задание и наше решение
```python
task = r'''Вычислить $\displaystyle \lim_{n \to \infty} (\sqrt{n^4 - 3n^2 + 2} - \sqrt{n^4 + 5n^2 + 1})$. Ответ запишите в виде десятичной дроби, округлив по правилам математического округления до 3 знаков после запятой.'''
sol = r'''Посмотрим на запись: там разность двух корней четвёртой степени. Когда n большое, каждый из них примерно как n^2, и прямой предел даёт что-то вроде «бесконечность минус бесконечность». Чтобы от этого избавиться, удобнее умножить и поделить на сумму этих корней — стандартный приём с сопряжённым.

Обозначу A = √(n^4 − 3n^2 + 2), B = √(n^4 + 5n^2 + 1). Тогда
A − B = (A − B)(A + B)/(A + B) = (A^2 − B^2)/(A + B).
Разность под корнем раскрывается просто:
A^2 − B^2 = (n^4 − 3n^2 + 2) − (n^4 + 5n^2 + 1) = −6n^2 + 1.
Значит,
√(n^4 − 3n^2 + 2) − √(n^4 + 5n^2 + 1) = (−6n^2 + 1) / [√(n^4 − 3n^2 + 2) + √(n^4 + 5n^2 + 1)].

Дальше вынесу общий крупный масштаб n^2 из-под корней в знаменателе:
√(n^4 − 3n^2 + 2) = n^2 √(1 − 3/n + 2/n^4),
√(n^4 + 5n^2 + 1) = n^2 √(1 + 5/n + 1/n^4).
Тогда общий знаменатель превращается в
n^2 [ √(1 − 3/n + 2/n^4) + √(1 + 5/n + 1/n^4) ].

Теперь можно сократить на n^2:
(−6n^2 + 1) / { n^2 [ … ] } = (−6 + 1/n) / [ √(1 − 3/n + 2/n^4) + √(1 + 5/n + 1/n^4) ].

При n → ∞ обе скобки под корнями стремятся к единице, так что сумма внизу идёт к 2, а вверху остаётся просто −6. В итоге предел равен −6/2 = −3.

Округляя до трёх знаков после запятой:
−3.000

Ответ: −3.000'''
```

## Непосредственно получаем диапазоны
```python
intervals = web(task, sol)
```

## Выводим результат с разметкой цветом
```python
print(highlight_by_indices(sol, dia))
```

In [4]:
# task = r'''Вычислить $\displaystyle \lim_{n \to \infty} (\sqrt{n^4 - 3n^2 + 2} - \sqrt{n^4 + 5n^2 + 1})$. Ответ запишите в виде десятичной дроби, округлив по правилам математического округления до 3 знаков после запятой.'''
# sol = r'''Посмотрим на запись: там разность двух корней четвёртой степени. Когда n большое, каждый из них примерно как n^2, и прямой предел даёт что-то вроде «бесконечность минус бесконечность». Чтобы от этого избавиться, удобнее умножить и поделить на сумму этих корней — стандартный приём с сопряжённым.

# Обозначу A = √(n^4 − 3n^2 + 2), B = √(n^4 + 5n^2 + 1). Тогда
# A − B = (A − B)(A + B)/(A + B) = (A^2 − B^2)/(A + B).
# Разность под корнем раскрывается просто:
# A^2 − B^2 = (n^4 − 3n^2 + 2) − (n^4 + 5n^2 + 1) = −6n^2 + 1.
# Значит,
# √(n^4 − 3n^2 + 2) − √(n^4 + 5n^2 + 1) = (−6n^2 + 1) / [√(n^4 − 3n^2 + 2) + √(n^4 + 5n^2 + 1)].

# Дальше вынесу общий крупный масштаб n^2 из-под корней в знаменателе:
# √(n^4 − 3n^2 + 2) = n^2 √(1 − 3/n + 2/n^4),
# √(n^4 + 5n^2 + 1) = n^2 √(1 + 5/n + 1/n^4).
# Тогда общий знаменатель превращается в
# n^2 [ √(1 − 3/n + 2/n^4) + √(1 + 5/n + 1/n^4) ].

# Теперь можно сократить на n^2:
# (−6n^2 + 1) / { n^2 [ … ] } = (−6 + 1/n) / [ √(1 − 3/n + 2/n^4) + √(1 + 5/n + 1/n^4) ].

# При n → ∞ обе скобки под корнями стремятся к единице, так что сумма внизу идёт к 2, а вверху остаётся просто −6. В итоге предел равен −6/2 = −3.

# Округляя до трёх знаков после запятой:
# −3.000

# Ответ: −3.000'''
# prompts = {'decompose':prompt_decompose_solution}
# web = WebMarkingError(prompts)
# web(task, sol)